<h1> <center> DATA CLEANING </center> </h1>
<h3> Initial setup (dataset loading and complementary features) </h3>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from time import process_time
import numpy as np

class Summary:
    def __init__(self, data, *args):
        self._summary = data
        
    def get_descr(self, colname):
        
        """Returns the description of a specific variable"""
        
        return self._summary.loc[self._summary["Variable"] == colname, "Description"].values[0]
    
    def get_type(self, colname):
        
        """Returns the type of a specific variable"""
        
        return self._summary.loc[self._summary["Variable"] == colname, "Type"].values[0]
    
    def get_ifna(self, colname):
        
        """Returns whether a specific variable allows NA values"""
        
        return self._summary.loc[self._summary["Variable"] == colname, "Accepts NAs"].values[0]
    
    def get_cols_by_type(self, type):
        
        """Returns the names of the variables of a certain type. If you want the output
        to be a list, you must transform it to list (list(get_cols_by_type('int')))"""
        
        return self._summary.loc[self._summary["Type"] == type, "Variable"]

In [2]:
nov2014 = pd.read_csv("PAR-2014-Nov2-Nov29.csv")
nov2015 = pd.read_csv("PAR-2015-Nov1-Nov28.csv")
summary = Summary(pd.read_csv("VariablesDescr.csv", sep = ";", nrows = 15))

<h3> Appropiate data type conversion </h3>

In [3]:
#Change data types accordingly:

#Datetime
for var in list(summary.get_cols_by_type("datetime")):
    nov2014[var] = pd.to_datetime(nov2014[var])
    nov2015[var] = pd.to_datetime(nov2015[var])

#Categories
for var in list(summary.get_cols_by_type("factor")):
    nov2014[var] = nov2014[var].astype("category")
    nov2015[var] = nov2015[var].astype("category")

<h3> Filter rows with negative value in the variable "leadtime" </h3>

In [4]:
#Eliminate rows whose value in leadtime is negative

nov2014 = nov2014.loc[nov2014["leadtime"] >= 0,:]
nov2015 = nov2015.loc[nov2015["leadtime"] >= 0,:]

<h3> Extract day of the week from bookingday </h3>

In [5]:
#Extracting day of the week

t1 = process_time()

nov2014["bookingday"] = nov2014.iloc[:,0:1].applymap(lambda x: x.day_name())
nov2015["bookingday"] = nov2015.iloc[:,0:1].applymap(lambda x: x.day_name())

t2 = process_time()

print(f"Days extracted in {t2-t1} s")

Days extracted in 102.0 s


In [6]:
nov2015.iloc[0:10, [0,-1]]

,bookingdate,bookingday
0,2015-11-08,Sunday
1,2015-11-08,Sunday
2,2015-11-08,Sunday
3,2015-11-08,Sunday
4,2015-11-08,Sunday
5,2015-11-08,Sunday
6,2015-11-08,Sunday
7,2015-11-08,Sunday
8,2015-11-08,Sunday
9,2015-11-08,Sunday


<h3> Separating the different meanings of the lenghofstay variable </h3>

In [7]:
#We know any value lesser or equal than 0 corresponds to a type of transfer (losname variable). Just a quick check:

aggregate = nov2014.loc[:,["losname", "lengthofstay", "bookingdate"]].groupby(by = ["losname", "lengthofstay"], observed = True).count()
print("Nov2014:")
for index, value in aggregate.iterrows():
    if index[0] != "STAY":
        print(index)

aggregate = nov2015.loc[:,["losname", "lengthofstay", "bookingdate"]].groupby(by = ["losname", "lengthofstay"], observed = True).count()
print("\nNov2015:")
for index, value in aggregate.iterrows():
    if index[0] != "STAY":
        print(index)

Nov2014:
('RETURN_HOME', -3)
('SHORT_TRANSFER', -4)
('DWELLING_TRANSFER', -2)
('LONG_TRANSFER', 0)
('END_OF_TRIP', -1)
('TRANSIT', -6)
('DAY_TRIP', -5)

Nov2015:
('DWELLING_TRANSFER', -2)
('RETURN_HOME', -3)
('SHORT_TRANSFER', -4)
('END_OF_TRIP', -1)
('DAY_TRIP', -5)
('LONG_TRANSFER', 0)
('TRANSIT', -6)


In [8]:
def stay(los):
    return np.nan if los <= 0 else int(los)

def transfer(losname):
    return np.nan if losname == "STAY" else losname

In [9]:
nov2014["losclean"] = nov2014.iloc[:,8:9].applymap(stay)
nov2014["losnameclean"] = nov2014.iloc[:,9:10].applymap(transfer)

nov2015["losclean"] = nov2014.iloc[:,8:9].applymap(stay)
nov2015["losnameclean"] = nov2014.iloc[:,9:10].applymap(transfer)

In [10]:
nov2014.iloc[0:10, [8,9,16,17]]

,lengthofstay,losname,losclean,losnameclean
0,10,STAY,10.0,NaN
1,-3,RETURN_HOME,NaN,RETURN_HOME
2,5,STAY,5.0,NaN
3,-4,SHORT_TRANSFER,NaN,SHORT_TRANSFER
4,-2,DWELLING_TRANSFER,NaN,DWELLING_TRANSFER
5,-2,DWELLING_TRANSFER,NaN,DWELLING_TRANSFER
6,29,STAY,29.0,NaN
7,-4,SHORT_TRANSFER,NaN,SHORT_TRANSFER
8,-2,DWELLING_TRANSFER,NaN,DWELLING_TRANSFER
9,7,STAY,7.0,NaN


<h3> Number and types of cancellations per day </h3>

In [11]:
books2014 = nov2014.loc[:, ["bookingdate", "bookingsign"]].groupby(by = ["bookingdate", "bookingsign"]).size().unstack(fill_value = 0)
books2015 = nov2015.loc[:, ["bookingdate", "bookingsign"]].groupby(by = ["bookingdate", "bookingsign"]).size().unstack(fill_value = 0)

In [12]:
books2015.head(5)

bookingsign,FULL_CANCELLATION,NEW_BOOKING,PARTIAL_ADDITION,PARTIAL_CANCELLATION
bookingdate,,,,
2015-11-01,5147,26282,10167,25718
2015-11-02,18971,120904,34695,57434
2015-11-03,14621,116953,32692,54091
2015-11-04,12299,114267,32265,57528
2015-11-05,11446,110209,33352,68546


In [13]:
books2014["TOTAL_BOOKINGS"] = books2014["FULL_CANCELLATION"] + books2014["NEW_BOOKING"] + books2014["PARTIAL_ADDITION"] + books2014["PARTIAL_CANCELLATION"]
books2015["TOTAL_BOOKINGS"] = books2015["FULL_CANCELLATION"] + books2015["NEW_BOOKING"] + books2015["PARTIAL_ADDITION"] + books2015["PARTIAL_CANCELLATION"]
books2015.head(5)

bookingsign,FULL_CANCELLATION,NEW_BOOKING,PARTIAL_ADDITION,PARTIAL_CANCELLATION,TOTAL_BOOKINGS
bookingdate,,,,,
2015-11-01,5147,26282,10167,25718,67314
2015-11-02,18971,120904,34695,57434,232004
2015-11-03,14621,116953,32692,54091,218357
2015-11-04,12299,114267,32265,57528,216359
2015-11-05,11446,110209,33352,68546,223553


We will also get another dataset with the relative frequencies instead of the absolute ones:

In [14]:
relbooks2014 = books2014.copy()
relbooks2015 = books2015.copy()

In [15]:
for book in relbooks2014.columns[:-1]:
    relbooks2014[book] = relbooks2014[book] / relbooks2014["TOTAL_BOOKINGS"]
    relbooks2015[book] = relbooks2015[book] / relbooks2015["TOTAL_BOOKINGS"]
    
relbooks2014["TOTAL_BOOKINGS"] = [1] * len(relbooks2014)
relbooks2015["TOTAL_BOOKINGS"] = [1] * len(relbooks2015)

In [16]:
relbooks2014.head(10)

bookingsign,FULL_CANCELLATION,NEW_BOOKING,PARTIAL_ADDITION,PARTIAL_CANCELLATION,TOTAL_BOOKINGS
bookingdate,,,,,
2014-11-02,0.079125,0.385850,0.153633,0.381393,1
2014-11-03,0.088928,0.540762,0.134495,0.235816,1
2014-11-04,0.074042,0.514124,0.155740,0.256094,1
2014-11-05,0.059511,0.503101,0.156893,0.280495,1
2014-11-06,0.051753,0.486987,0.147386,0.313874,1
2014-11-07,0.057279,0.461649,0.151936,0.329136,1
2014-11-08,0.064798,0.421071,0.124905,0.389226,1
2014-11-09,0.083729,0.409499,0.122861,0.383911,1
2014-11-10,0.086779,0.531997,0.131859,0.249365,1


In [17]:
nov2014.to_csv("Nov2014.csv", sep = ";", index = False)
nov2015.to_csv("Nov2015.csv", sep = ";", index = False)

books2014.to_csv("BookingsPerDay2014.csv", sep = ";")
books2015.to_csv("BookingsPerDay2015.csv", sep = ";")

relbooks2014.to_csv("RelativeBookingsPerDay2014.csv", sep = ";")
relbooks2015.to_csv("RelativeBookingsPerDay2015.csv", sep = ";")

In [19]:
data = {}

for cat in np.unique(nov2014.paxprofile):
    data[cat] = nov2014.iloc[:,7:8].applymap(lambda x: x == cat).values

In [20]:
data

{'BUSINESS': array([[False],
        [False],
        [False],
        ...,
        [False],
        [False],
        [False]]),
 'GROUP': array([[False],
        [False],
        [False],
        ...,
        [False],
        [False],
        [False]]),
 'LEISURE': array([[ True],
        [ True],
        [ True],
        ...,
        [ True],
        [ True],
        [ True]]),
 'VFR': array([[False],
        [False],
        [False],
        ...,
        [False],
        [False],
        [False]])}